In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cp -r /content/drive/MyDrive/dataset /content/dataset


In [ ]:
!ls /content


dataset  drive	sample_data


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 6
DATASET_DIR = "/content/dataset"


In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    DATASET_DIR + "/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_gen.flow_from_directory(
    DATASET_DIR + "/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 12000 images belonging to 6 classes.
Found 2406 images belonging to 6 classes.


In [ ]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # transfer learning


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 932s 2s/step - accuracy: 0.7822 - loss: 0.6264 - val_accuracy: 0.8408 - val_loss: 0.4502
Epoch 2/5
 20/375 ━━━━━━━━━━━━━━━━━━━━ 12:38 2s/step - accuracy: 0.9338 - loss: 0.1700

KeyboardInterrupt: 

In [ ]:
model.save("cabai_mobilenetv2_base.h5")


In [ ]:
cp cabai_mobilenetv2_base.h5 /content/drive/MyDrive/


In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/cabai_mobilenetv2_base.h5")


In [ ]:
for layer in model.layers:
    layer.trainable = True

for layer in model.layers[:130]:
    layer.trainable = False


In [ ]:
import tensorflow as tf

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=1
)


In [ ]:
model.save("cabai_mobilenetv2_best.h5")


In [ ]:
!cp cabai_mobilenetv2_best.h5 /content/drive/MyDrive/


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/cabai_mobilenetv2_best.h5")


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


In [ ]:
with open("cabai_mobilenetv2.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
!cp cabai_mobilenetv2.tflite /content/drive/MyDrive/


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Load model terbaik sebelumnya dari Drive
model = load_model("/content/drive/MyDrive/cabai_mobilenetv2_best.h5")

# Fine-tuning (freeze sebagian layer)
for layer in model.layers:
    layer.trainable = True

for layer in model.layers[:130]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/cabai_mobilenetv2_best.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stop, checkpoint, reduce_lr]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9554 - loss: 0.1256
Epoch 1: val_accuracy improved from -inf to 0.82835, saving model to /content/drive/MyDrive/cabai_mobilenetv2_best.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 1051s 3s/step - accuracy: 0.9554 - loss: 0.1255 - val_accuracy: 0.8283 - val_loss: 0.8527 - learning_rate: 1.0000e-05
Epoch 2/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9714 - loss: 0.0740
Epoch 2: val_accuracy did not improve from 0.82835
375/375 ━━━━━━━━━━━━━━━━━━━━ 1016s 3s/step - accuracy: 0.9714 - loss: 0.0740 - val_accuracy: 0.8171 - val_loss: 0.9530 - learning_rate: 1.0000e-05
Epoch 3/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9757 - loss: 0.0705
Epoch 3: val_accuracy did not improve from 0.82835
375/375 ━━━━━━━━━━━━━━━━━━━━ 1015s 3s/step - accuracy: 0.9757 - loss: 0.0705 - val_accuracy: 0.8213 - val_loss: 0.8832 - learning_rate: 1.0000e-05
Epoch 4/30
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9783 - loss: 0.0599
Epoch 4: val_accuracy did not improve from 0.82835

Epoch 4: ReduceLROnPlateau reducing learning rate to 2.9999999242136253e-06.
375/375 ━━━━━━━━━━━━━━━━━━━━ 1009s 3s/step - accuracy: 0.9783 - loss: 0.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/cabai_mobilenetv2_best.h5")


In [ ]:
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[-25:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-6),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/cabai_mobilenetv2_best.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9837 - loss: 0.0436
Epoch 1: val_accuracy improved from -inf to 0.81796, saving model to /content/drive/MyDrive/cabai_mobilenetv2_best.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 973s 3s/step - accuracy: 0.9838 - loss: 0.0436 - val_accuracy: 0.8180 - val_loss: 0.9527
Epoch 2/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9877 - loss: 0.0358
Epoch 2: val_accuracy improved from 0.81796 to 0.82294, saving model to /content/drive/MyDrive/cabai_mobilenetv2_best.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 980s 3s/step - accuracy: 0.9877 - loss: 0.0358 - val_accuracy: 0.8229 - val_loss: 0.9217
Epoch 3/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9886 - loss: 0.0308
Epoch 3: val_accuracy improved from 0.82294 to 0.82793, saving model to /content/drive/MyDrive/cabai_mobilenetv2_best.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 1004s 3s/step - accuracy: 0.9886 - loss: 0.0308 - val_accuracy: 0.8279 - val_loss: 0.8361
Epoch 4/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9884 - loss: 0.0306
Epoch 4: val_accuracy did not improve from 0.82793
375/375 ━━━━━━━━━━━━━━━━━━━━ 962s 3s/step - accuracy: 0.9884 - loss: 0.0306 - val_accuracy: 0.8267 - val_loss: 0.8560
Epoch 5/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9892 - loss: 0.0283
Epoch 5: val_accuracy improved from 0.82793 to 0.82959, saving model to /content/drive/MyDrive/cabai_mobilenetv2_best.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 988s 3s/step - accuracy: 0.9892 - loss: 0.0283 - val_accuracy: 0.8296 - val_loss: 0.8605
Epoch 6/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9903 - loss: 0.0255
Epoch 6: val_accuracy did not improve from 0.82959
375/375 ━━━━━━━━━━━━━━━━━━━━ 958s 3s/step - accuracy: 0.9903 - loss: 0.0255 - val_accuracy: 0.8283 - val_loss: 0.8415
Epoch 7/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9913 - loss: 0.0236
Epoch 7: val_accuracy improved from 0.82959 to 0.83915, saving model to /content/drive/MyDrive/cabai_mobilenetv2_best.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 955s 3s/step - accuracy: 0.9913 - loss: 0.0236 - val_accuracy: 0.8392 - val_loss: 0.7518
Epoch 8/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9910 - loss: 0.0253
Epoch 8: val_accuracy did not improve from 0.83915
375/375 ━━━━━━━━━━━━━━━━━━━━ 986s 3s/step - accuracy: 0.9910 - loss: 0.0253 - val_accuracy: 0.8292 - val_loss: 0.8947
Epoch 9/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9906 - loss: 0.0259
Epoch 9: val_accuracy did not improve from 0.83915
375/375 ━━━━━━━━━━━━━━━━━━━━ 980s 3s/step - accuracy: 0.9906 - loss: 0.0258 - val_accuracy: 0.8242 - val_loss: 0.9283
Epoch 10/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9911 - loss: 0.0250
Epoch 10: val_accuracy did not improve from 0.83915
375/375 ━━━━━━━━━━━━━━━━━━━━ 959s 3s/step - accuracy: 0.9911 - loss: 0.0250 - val_accuracy: 0.8321 - val_loss: 0.8300
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 7.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

# 1. Load BEST model (epoch 7)
model = load_model("/content/drive/MyDrive/cabai_mobilenetv2_best.h5")

# 2. Freeze lebih banyak layer (kurangi overfitting)
for layer in model.layers:
    layer.trainable = True

for layer in model.layers[:140]:
    layer.trainable = False

# 3. Compile ulang
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

# 4. Callback FINAL
early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=4,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/cabai_mobilenetv2_85.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# 5. Training ringan (FINAL PUSH)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9716 - loss: 1.3755
Epoch 1: val_accuracy improved from -inf to 0.84040, saving model to /content/drive/MyDrive/cabai_mobilenetv2_85.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 987s 3s/step - accuracy: 0.9716 - loss: 1.3754 - val_accuracy: 0.8404 - val_loss: 1.9494
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9648 - loss: 1.1120
Epoch 2: val_accuracy improved from 0.84040 to 0.84248, saving model to /content/drive/MyDrive/cabai_mobilenetv2_85.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 986s 3s/step - accuracy: 0.9648 - loss: 1.1118 - val_accuracy: 0.8425 - val_loss: 1.6496
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9610 - loss: 0.9074
Epoch 3: val_accuracy did not improve from 0.84248
375/375 ━━━━━━━━━━━━━━━━━━━━ 984s 3s/step - accuracy: 0.9610 - loss: 0.9073 - val_accuracy: 0.8350 - val_loss: 1.4358
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9548 - loss: 0.7981
Epoch 4: val_accuracy did not improve from 0.84248
375/375 ━━━━━━━━━━━━━━━━━━━━ 972s 3s/step - accuracy: 0.9548 - loss: 0.7980 - val_accuracy: 0.8275 - val_loss: 1.2902
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9500 - loss: 0.7393
Epoch 5: val_accuracy did not improve from 0.84248
375/375 ━━━━━━━━━━━━━━━━━━━━ 973s 3s/step - accuracy: 0.9500 - loss: 0.7393 - val_accuracy: 0.8204 - val_loss: 1.2049
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9465 - loss: 0.7162
Epoch 6: val_accuracy did not impro

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(
    "/content/drive/MyDrive/cabai_mobilenetv2_85.h5",
    compile=False
)


In [ ]:
!pip uninstall -y tensorflow keras tf-keras jax jaxlib ml_dtypes
!pip install --no-deps tensorflow==2.16.2 tf-keras==2.16.0


Found existing installation: tensorflow 2.16.2
Uninstalling tensorflow-2.16.2:
  Successfully uninstalled tensorflow-2.16.2
Found existing installation: keras 3.13.0
Uninstalling keras-3.13.0:
  Successfully uninstalled keras-3.13.0
Found existing installation: tf_keras 2.16.0
Uninstalling tf_keras-2.16.0:
  Successfully uninstalled tf_keras-2.16.0
Found existing installation: ml-dtypes 0.3.1
Uninstalling ml-dtypes-0.3.1:
  Successfully uninstalled ml-dtypes-0.3.1
  Using cached tensorflow-2.16.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached tf_keras-2.16.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.16.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (590.8 MB)
Using cached tf_keras-2.16.0-py3-none-any.whl (1.7 MB)


In [ ]:
import tensorflow as tf
import tf_keras

print("TF:", tf.__version__)
print("TF-Keras:", tf_keras.__version__)


/usr/local/lib/python3.12/dist-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


TF: 2.16.2
TF-Keras: 2.16.0


In [ ]:
# KERAS → TFLITE LEGACY CONVERTER

import tensorflow as tf
import os

MODEL_PATH = "/content/drive/MyDrive/cabai_mobilenetv2_85.h5"
TFLITE_PATH = "/content/drive/MyDrive/cabai_mobilenetv2_85.tflite"

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("MODEL .h5 TIDAK DITEMUKAN")

print("Model ditemukan, lanjut...")

model = tf.keras.models.load_model(
    MODEL_PATH,
    compile=False
)

print("Model berhasil diload")
model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS
]

# Mode lama
converter.experimental_new_converter = False
converter.experimental_enable_resource_variables = False

tflite_model = converter.convert()
print("Convert sukses")

with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print("TFLite tersimpan di:", TFLITE_PATH)

interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("\n=== INTERPRETER TEST ===")
print("INPUT :", input_details)
print("OUTPUT:", output_details)

print("\nSELESAI — MODEL SIAP ANDROID")


Model ditemukan, lanjut...
Model berhasil diload


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 1,204,742 (4.60 MB)

 Non-trainable params: 1,217,984 (4.65 MB)

Convert sukses
TFLite tersimpan di: /content/drive/MyDrive/cabai_mobilenetv2_85.tflite

=== INTERPRETER TEST ===
INPUT : [{'name': 'input_layer', 'index': 183, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
OUTPUT: [{'name': 'Identity', 'index': 0, 'shape': array([1, 6], dtype=int32), 'shape_signature': array([1, 6], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

SELESAI — MODEL SIAP ANDROID
